# MAUP exercise: Putting demographic and election data together

_Communication: At your discretion -- feel free to use Slack or Zoom to communicate, but otherwise, work independently. MGGG staff will be available as usual._

To understand the interaction between voting, race and representation, we need to deal with data at various spatial scales and in various spatial units. For example, most demographic data is presented at the level of __census blocks__ every decennial Census. Citizen voting age population (CVAP) data, however, is not collected during the decennial census, and is instead collected by the American Community Survey (ACS) and presented at the level of __block groups__, one step higher in the hierarchy of Census spatial data. Finally, voting data is presented at the level of __precincts__. As you saw in Project 1 and throughout last week, precincts are often hard to get hold of and don't always match very well with census geography.

Getting demographic, CVAP, and election data all on the same set of units requires making thoughtful choices and using the right computational tools. In this exercise, we will use MAUP, a package developed by the MGGG, to perform some of these types of operations. 

The examples in this notebook come from __Lowell MA__. The MGGG recently completed a research project in Lowell looking at how electoral reform (e.g. ranked choice voting, single-member districts or multi-member districts) could help minority groups (in this case, Asian and Hispanic voters) achieve more representation on the city council. You'll learn more about the Lowell project tomorrow evening. For now, we're zooming in on a particular part of our analysis: we needed to use demographic data (including CVAP from the ACS) as well as past election data, and analyze these data at the same spatial scale. In this exercise you will replicate some of this data processing using real data from Lowell.

In this exercise, we are going to:


*   Use MAUP to _disaggregate_ CVAP data from block groups to blocks.
*   Use MAUP to _aggregate_ demographic data from blocks up to precincts.
*   Use MAUP to _assign_ blocks to districts/wards.

Everything you need to know for the exercise is explained below, but you can also check out the MAUP documentation at https://github.com/mggg/maup.

![](https://drive.google.com/uc?export=view&id=1JjziUCOCB_dJjBJoQNdksSlMx3T1HBb_)

In [ ]:
#this installs geopandas and maup, which we will need for this exercise
!pip install geopandas
!pip install maup

In [ ]:
#go and get some data for the exercise from github
! git clone https://github.com/thomasweighill/maup-exercise.git

In [ ]:
#ignore this - just to suppress some unhelpful warnings
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

## Part 1: Disaggregating CVAP from block groups to blocks
Let's get the CVAP data (at the block group level) down to the block level. This will not be the _truth_, of course, because block groups are bigger than blocks, but we can make a best approximation.

### Exploring block and block group data

Let's load block group and block shapefiles, and see what fields they have.

In [ ]:
import geopandas as gpd
import maup
import matplotlib.pyplot as plt

In [ ]:
#load a block group shapefile
block_groups = gpd.read_file("maup-exercise/lowell_bgs.shp")

In [ ]:
#let's look at the block group data a bit
block_groups.plot()
print("There are {} rows in the shapefile".format(len(block_groups)))
print("The block group shapefile has columns: ", block_groups.columns)

In [ ]:
#load a block shapefile
blocks = gpd.read_file("maup-exercise/lowell_blocks.shp")

In [ ]:
#let's look at the data a bit
blocks.plot()
print("There are {} rows in the shapefile".format(len(blocks)))
print("The block group shapefile has columns: ", blocks.columns)

Notice that the blocks shapefile doesn't have CVAP, but the block group shapefile does. We want to use the block group CVAP data to get approximate CVAP on blocks. To give you some intuition for how this works, let's look at just one block group.

In [ ]:
#pull out one block group as a demonstration
example_block_group = block_groups[42:43]
example_block_group.plot()

Now that we have our example block group, let's plot its boundary on top of our blocks.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10)) #makes the figure nice and big
blocks.plot(ax=ax)
example_block_group.boundary.plot(ax=ax,color='yellow', linewidth=10)

As you can see, our block group contains many blocks. One simple idea would be to divide the CVAP in the block group equally among all the blocks in it. Take a minute and think for yourself whether this is a good idea before proceeding on.

In [ ]:
#plot the same plot, but with population shown for blocks
fig, ax = plt.subplots(figsize=(20,10)) #makes the figure nice and big
blocks.plot(ax=ax, column='TOTPOP', cmap='Blues',legend=True,edgecolor='black')
example_block_group.boundary.plot(ax=ax,color='yellow', linewidth=8)

The plot above shows the population by block. Most of the population is in just one block! It's therefore not realistic to share the CVAP equally among all the blocks in our block group. Instead, we should give CVAP to each block according to its population. This is what we refer to as _proration_ or _disaggregation_.

### Using MAUP for disaggregation

Fortunately, the MAUP package does this process for us. Let's give it a shot. The first thing we must ALWAYS do is make sure the two shapefiles (blocks and block_groups) are in the same projection!

In [ ]:
#let's find out which projection the block shapefile is in
blocks.crs

In [ ]:
#let's reproject them to a local MA projection (ignore the warning)
blocks = blocks.to_crs({"init":"epsg:2249"})

In [ ]:
#project block_groups to same projection as blocks
block_groups = block_groups.to_crs(blocks.crs)

In [ ]:
#this assigns blocks to block groups than contain it (this can take a while with big shapefiles!)
#fill in the ellipses with the geodataframes for the smaller units and the bigger units respectively
assignment = maup.assign(..., ...)

In [ ]:
#this gets us the weighting for each block by population
weights = blocks.TOTPOP / assignment.map(block_groups.TOTPOP)

In [ ]:
#these are the columns we want to prorate to blocks
columns = ['CVAP', 'CVAPHISP', 'CVAPWHITE', 'CVAPASIAN']

In [ ]:
#now we do the proration from block groups onto blocks
prorated = maup.prorate(assignment, block_groups[columns], weights)
#remember to put the new data onto the blocks
blocks[columns] = prorated

That's it! We've put CVAP on blocks. Let's take a look at our new data. Let's plot blocks showing the percent of the population that are voting age citizens.

In [ ]:
blocks.plot(column=blocks.CVAP.divide(blocks.TOTPOP),cmap='Blues', legend=True)

In [ ]:
#let's save our file so we can open it later
blocks.to_file("lowell_blocks_with_new_data.shp")

*Extra challenge (optional):* By making more plots, find some other demographic stats that look like they are correlated (or anti-correlated) with the percent of the population that are voting age citizens.

### Sanity checks!

Let's do at least one check to make sure we didn't mess up.

In [ ]:
#the total CVAP using the block group shapefile
print("TOTAL CVAP using block groups:", block_groups['CVAP'].sum())

In [ ]:
#the total CVAP using the block shapefile
print("TOTAL CVAP using blocks:", blocks['CVAP'].sum())

There may be some rounding error, but the numbers should be roughly the same.

## Part 2: Aggregating demographic data from blocks to precincts
Let's get demographic data from census blocks up to precincts. If the blocks fit perfectly inside the precincts, this should be 100% accurate (no approximation required). 

### Exploring precinct and block data

Let's load some voting precincts, and take a look at how they look over blocks.

In [ ]:
#load a precinct shapefile
precincts = gpd.read_file("maup-exercise/lowell_precincts.shp")

In [ ]:
#reproject the precincts (ignore warnings again)
precincts = precincts.to_crs({"init":"epsg:2249"})

In [ ]:
#let's look at precincts over blocks
fig, ax = plt.subplots(figsize=(20,10)) #makes the figure nice and big
blocks.plot(ax=ax)
precincts.boundary.plot(ax=ax,color='yellow', linewidth=3)

### Using MAUP to aggregate demographic data up to precincts

We want to add demographic data to our precinct shapefile. This time, we are going from smaller units (blocks) to larger units (precincts), so we are going to just add up all the block data in each precinct. Sometimes a block touches multiple precincts, which makes things hard. Fortunately, MAUP handles that for us by choosing the precinct with the highest overlap.

In [ ]:
#this gives us an assignment for each block to the best precinct for it
#fill in the ellipses with the smaller units and the bigger units respectively
assignment = maup.assign(..., ...)

In [ ]:
#these are the columns we want to add to our precincts
variables = ['TOTPOP', 'NH_WHITE',
       'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE',
       'HISP', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER',
       'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP',
       'NHPIVAP', 'OTHERVAP', '2MOREVAP', 'CVAP', 'CVAPHISP', 'CVAPWHITE',
       'CVAPASIAN']

In [ ]:
#now we need to add up the data for each precinct
precincts[variables] = blocks[variables].groupby(assignment).sum()

That's it! We've used blocks to put demographic data on precincts. Let's take a quick look at our new data. We'll plot the precincts colored by how much of their CVAP is Hispanic.

In [ ]:
precincts.plot(column=precincts.CVAPHISP.divide(precincts.CVAP),cmap='Blues',legend=True)

In [ ]:
#let's save our file
precincts.to_file("lowell_precincts_with_new_data.shp")

### Sanity checks!

In [ ]:
#the total CVAP using the block group shapefile
print("TOTAL CVAP using blocks:", blocks['CVAP'].sum())
#the total CVAP using the block group shapefile
print("TOTAL CVAP using precincts:", precincts['CVAP'].sum())

## Part 3: Assigning blocks to districts
We can also use MAUP to assign blocks to districts (or wards, as they are sometimes called). As an example, let's look at some sample districts proposed by the MGGG for Lowell.  

### Exploring district data

In [ ]:
#the sample districts as a shapefile
districts = gpd.read_file('maup-exercise/lowell_sample_plan.shp')

In [ ]:
#taking a look at the districts on a map
districts.plot(cmap='tab10')

In [ ]:
#taking a look at the data itself
districts.head()

### Using MAUP to assign blocks to districts

Assigning blocks to districts works roughly the same as assigning blocks to precincts. We will make one small change so that when the district names are put on the block data, MAUP uses the actual names of the districts and not just the index. In other words, we want to see "District 7" instead of just a number.

In [ ]:
#always reproject! 
districts = districts.to_crs(blocks.crs)

In [ ]:
#let's assign blocks to districts, using 'District' as the index column
assignment = maup.assign(blocks, districts.set_index('District')) 

In [ ]:
#put the District field onto the block data
blocks['District'] = assignment

### Sanity checks and computing demographic stats of districts

In [ ]:
#plot blocks colored by district to see if we did a good job (compare to the picture above)
#note that the legend shows the District names, not numbers!
blocks.plot(column='District', legend=True)

In [ ]:
#we can now compute demographic stats, like what percent Hispanic CVAP each district has
district_demographics = blocks.groupby('District').sum()[['CVAP', 'CVAPASIAN','CVAPHISP']]
district_demographics.head()

In [ ]:
#let's calculate the percent of CVAP that is either Hispanic or Asian 
(district_demographics['CVAPASIAN']+district_demographics['CVAPHISP']).divide(district_demographics['CVAP'])

Do you see three districts over 40%? This map was designed to have three districts with high 'coalition' (Asian + Hispanic) voting populations.

# Summary

Let's summarize what we have done in this exercise. Our goal was to get demographic data onto precincts. Some of it was at the block level and some was at the block group level. We therefore _disaggregated_ CVAP from block groups to blocks and then _aggregated_ the CVAP data and the data that was already on the blocks up to precincts. Here's a picture of what we did:

![](https://drive.google.com/uc?export=view&id=1JjziUCOCB_dJjBJoQNdksSlMx3T1HBb_)

# Next steps

As a next step (if you have time), why don't you try doing all of this for Everett MA, another city where the MGGG analyzed electoral reform in the context of minority representation. You can go and change the code above directly, or write/paste your own code in the blocks below. Jupyter notebooks (that's what this code is in) are designed for play!

Some prepared files are already available:
- maup-exercise/everett_bgs.shp (block groups)
- maup-exercise/everett_blocks.shp (blocks)
- maup-exercise/everett_precincts.shp (precincts)
- maup-exercise/everett_city_wards.shp (current districts)

You may have to change the column names as you go (CVAPHISP may be called HCVAP, for example), but that's part of the experience.

**Possible goal:** Find the non-white CVAP percentages for the current Everett districts.